# Training YOLOv4 with Custom Dataset from Open Images Database v6 (OIDv6)

This Jupyter Notebook is based on [YOLOv4: A step-by-step guide for Custom Data Preparation with Code](https://techylem.com/yolov4-guide-with-code/) with information of [TRAIN A CUSTOM YOLOv4 OBJECT DETECTOR (Using Google Colab)](https://medium.com/analytics-vidhya/train-a-custom-yolov4-object-detector-using-google-colab-61a659d4868).

It also uses the [OIDv6 tool](https://github.com/DmitryRyumin/OIDv6) to download the dataset from Google's [Open Images Dataset](https://storage.googleapis.com/openimages/web/index.html). And its helper scripts are inspired from [OIDv4_ToolKit](https://github.com/ahsan44411/OIDv4_ToolKit)

Details about darknet customization options to make them work better on Google's Colab are available at

* [Darknet FAQ](https://www.ccoderun.ca/programming/darknet_faq/)
* [CFG Parameters in the [net] section](https://github.com/AlexeyAB/darknet/wiki/CFG-Parameters-in-the-%5Bnet%5D-section)

Information about Yolov4-tiny training and when to stop training at [AlexeyAB/darknet](https://github.com/AlexeyAB/darknet#how-to-train-tiny-yolo-to-detect-your-custom-objects).

## Prepare Custom Dataset

### Download your dataset

Install oidv6 to download your dataset

`$ pip3 install --user oidv6`

Create a classes.txt where each line is a class you want to download

You can download the classes names from https://storage.googleapis.com/openimages/web/download.html under section "Annotations and metadata"; row "Metadata" and pressing the button "Class Names".

Think about how many pictures for training, validation and testing you want.
Example: train - 300, validation - 75, test - 10.

Where 300 images for training are the 80\% and 75 images for validation 75 are 20\% of a dataset of 375 per class.

Note that the amount of images per class available on the OID is not the same. Some classes have under 100 images or less.

Download your dataset for training with

`$ oidv6 downloader en --type_data train --classes ./classes.txt --limit 300 --multi_classes`

the dataset for validation with

`$ oidv6 downloader en --type_data validation --classes ./classes.txt --limit 75 --multi_classes`

and the dataset for testing with

`$ oidv6 downloader en --type_data test --classes ./classes.txt --limit 10 --multi_classes`

Note that with `--limit NN` you specify how many images you want for each class of the dataset

`--multi_classes` will put all pictures on one directory ("train", "validation", "test" according to your `--type_data` selected) and all annotations labels inside one directory named `labels` inside your dataset directory. Without this option, each class' pictures will be downloaded into individual directories in which each one of them will have an individual `labels` directory with all annotations label files.

**Some classes might not have any picture in the OID** a red warning will be shown during the download to let you know about these classes without pictures.


### Adapt your Dataset to YOLOv4 format

The annotation files have this format

`label_name x1 y1 x2 y2`

Which is not the annotation label format of YOLO which is

`label_index box_center_x box_center_y box_width box_heigth`

Where

**label_index**: is the index of the label inside `classes.txt`

**box_center_x**: is the x value* of the center of the bouding box

**box_center_y**: is the y value* of the center of the bouding box

**box_width**: is the width* of the bouding box

**box_heigth**: is the height* of the bouding box

And that such coordinates, width and height are represented by a float number between \[0, 1\] where 0 is the origin and 1 is the max. width or max. height.

Also, the darknet tool will ask for a list of all the file paths of the images for training and validation.

All this is solved with the script `prepare_dataset.py`.

It will also generate the configuration file `objects.txt` needed for darknet.

Run `$ python3 prepare_dataset.py`

Now, inside the OIDv6 directory you will have `multidata` directory with your dataset, a copy of `classes.txt` and the configuration file `objects.txt`. 

Zip the 3 of them up into a `dataset.zip` file, and **upload it to your Google Drive** inside a directory structure like this: **`Training/Data/dataset.zip`**


### Mount Google Drive

Let's mount your Google Drive into Colab's runtime

In [ ]:
from google.colab import drive
drive.mount('/drive')

Copy the dataset into the filesystem

In [ ]:
!cp -r '/drive/MyDrive/Training/Data/dataset.zip' /content

And unzip it

In [ ]:
!unzip '/content/dataset.zip' -d '/content'

## Prepare Darknet Tool

### Download darknet source code

Clone darknet project. This is a framework/tool to train and customize several YOLO versions.

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Modify makefile to work with the GPU and OpenCV

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

Check if CUDA Compiler is installed

In [ ]:
!/usr/local/cuda/bin/nvcc --version

Compile darknet

In [ ]:
!make

### (Optional) Test darknet

Download pre-trained YOLOv4 weights

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

Defining function to show output images

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
def imShow(path):
  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

Run the Object Recognition Model on test image.

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/person.jpg
imShow('predictions.jpg')

### Train with Custom Data

Copy and customize `yolov4-tiny-custom.cfg`

In [ ]:
!cp -r '/content/darknet/cfg/yolov4-tiny-custom.cfg' '/content/darknet/cfg/yolov4-tiny-obj.cfg'

You can run the script `customize_yolov4_tiny.py` to automatically adapt `yolov4-tiny-obj.cfg` to the current dataset.

Changes that it will make:

```
[net]
max_batches = (# of Classes * 2000)
steps = (80% of max_batches), (90% of max_batches)

[convolutional]
filters = ( (# of Classes + 5) * 3 )
[yolo]
classes = # of Classes

[convolutional]
filters = ( (# of Classes + 5) * 3 )
[yolo]
classes = # of Classes

[convolutional]
filters = ( (# of Classes + 5) * 3 )
[yolo]
classes = # of Classes
```

Download the pre-trained weights

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

Download dataset and configuration

In [ ]:
!git clone --depth 1 https://github.com/sergiodl-naist/yolov4-oidv6-test

Begin training

In [ ]:
# %%capture
!./darknet detector train \
  yolov4-oidv6-test/object.data \
  yolov4-oidv6-test/yolov4-custom.cfg \
  yolov4.conv.137 \
  -dont_show \
  -map

Or restart training

In [ ]:
!./darknet detector train \
  yolov4-oidv6-test/obj.data \
  yolov4-oidv6-test/yolov4-custom.cfg \
  /drive/MyDrive/TrainingBackup/yolov4-obj_last.weights \
  -dont_show